In [1]:
from useful_scit.imps import (pd,np,xr,za,mpl,plt,sns, pjoin, os,glob,dt,
    sys,ucp,log, splot, crt,axsplot)
import flexpart_management.modules.constants as co
import flexpart_management.modules.flx_array as fa

reload


In [5]:
print(sys.argv)

['/homeappl/home/aliagadi/appl_taito/miniconda3/envs/b36backup/lib/python3.6/site-packages/ipykernel_launcher.py',
 '-f',
 '/homeappl/home/aliagadi/.local/share/jupyter/runtime/kernel-72f8fbc3-0ad8-436c-be80-a5c0fad941f2.json']

In [ ]:
PATH_IN = sys.argv[1]
PATH_OUT = sys.argv[2]
DOM = sys.argv[3]

In [26]:
# PATH_IN = '/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpart_management_data/'+\
# 'runs/run_2019-09-25_15-25-01_/2018-01-01/'
# PATH_OUT = '/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpart_management_data/'+\
# 'runs/run_2019-09-25_15-25-01_/check_plots'
# DOM = 1

In [12]:
path = os.path.join(PATH_IN,f'flx*_d0{DOM}*.nc')

In [13]:
os.makedirs(PATH_OUT,exist_ok=True)

In [14]:
ls_files = glob.glob(path)

In [15]:
ls_files.sort()

In [16]:
P='PATH'
N='NAME'
D='DOM'
O='OUT'
ND = 'NAME_DAY'
DO = 'DIR_OUT'
df = pd.DataFrame(ls_files,columns=[P])

In [22]:
df[N]=df[P].apply(lambda p: os.path.basename(p))
df[ND]=df.apply(lambda r: os.path.basename(os.path.dirname(r[P])),axis=1)
df[O]=df.apply(lambda r: os.path.join(PATH_OUT,r[ND],r[N]+'.jpg'),axis=1)
df[DO]=df.apply(lambda r: os.path.dirname(r[O]),axis=1)
df[D]=df[N].str.extract('_d0(.)').astype(int)
df

,PATH,NAME,NAME_DAY,OUT,DIR_OUT,DOM
0,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,flxout_d01_20171228_000000.nc,2018-01-01,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,1
1,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,flxout_d01_20171228_010000.nc,2018-01-01,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,1
2,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,flxout_d01_20171228_020000.nc,2018-01-01,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,1
3,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,flxout_d01_20171228_030000.nc,2018-01-01,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,1
4,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,flxout_d01_20171228_040000.nc,2018-01-01,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,1
...,...,...,...,...,...,...
115,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,flxout_d01_20180101_190000.nc,2018-01-01,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,1
116,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,flxout_d01_20180101_200000.nc,2018-01-01,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,1
117,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,flxout_d01_20180101_210000.nc,2018-01-01,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,1
118,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,flxout_d01_20180101_220000.nc,2018-01-01,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpa...,1


In [23]:
_dn = df[ND]
_dn = _dn.drop_duplicates()
_dd = df[D]
_dd = _dd.drop_duplicates()

In [29]:
for dom in _dd:
    for day in _dn:
        dfs = df[(df[ND]==day)&(df[D]==dom)]
        out = f'{dfs[DO].iloc[0]}_0{dom}.jpg'
        print(out)
        ax = axsplot()
        try:
            ds =xr.open_mfdataset(dfs.iloc[:][P],concat_dim=co.TIME,combine='nested', preprocess=fa.convert_ds_time_format)
            ds.sum(fa.get_dims_complement(ds,co.TIME))[co.CONC].plot(ax=ax)
            
        except: pass
        ax.set_title(f'dom:{dom} - day:{day}')
        
        ax.figure.tight_layout()
        ax.figure.savefig(out)
        plt.close(ax.figure)
        print('done')
        


/homeappl/home/aliagadi/wrk/DONOTREMOVE/flexpart_management_data/runs/run_2019-09-25_15-25-01_/check_plots/2018-01-01_01.jpg
